<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium
from geopy.distance import geodesic

In [54]:
# Assuming 'trips.xlsx' is uploaded to your Colab environment
df_trips = pd.read_excel('trips.xlsx', header=0)  # header=0 means the first row is the header
df_trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed_gl,Ignition
2813,77403 (М947ОК67) Renault Logan,18.02.2025 13:51:48,"55,757661°, 37,827241°",28,+
1760,77403 (М947ОК67) Renault Logan,17.02.2025 22:59:00,"55,74506°, 37,813948°",19,+


In [55]:
# Split the 'Coordinates' column into 'Longitude' and 'Latitude'
df_trips['Сoordinates'] = df_trips['Сoordinates'].str.replace('°', '').str.replace(',', '.')
df_trips[['Longitude', 'Latitude']] = df_trips['Сoordinates'].str.split('. ', expand=True)
df_trips.drop(['Сoordinates'], axis=1, inplace=True)

# Convert 'Day_time' to datetime objects
df_trips['Day_time'] = pd.to_datetime(df_trips['Day_time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')

df_trips['Ignition'] = df_trips['Ignition'].str.replace('-', '0').str.replace('+', '1')

df_trips = df_trips.sort_values(['Car_ID', 'Day_time'])  # Сортируем по времени

df_trips.sample(2)

,Car_ID,Day_time,Speed_gl,Ignition,Longitude,Latitude
1944,77403 (М947ОК67) Renault Logan,2025-02-17 23:46:22,77,1,55.730476,37.808621
1068,77403 (М947ОК67) Renault Logan,2025-02-17 14:29:27,31,1,55.74999,37.831951


In [56]:
# Преобразование поездок в геометрию
df_trips['geometry'] = df_trips.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
df_trips = gpd.GeoDataFrame(df_trips, geometry='geometry', crs="EPSG:4326")

In [57]:
# Загрузка данных о Выделенных полосах
gdf_bus_lanes = gpd.read_file("bus-lanes.geojson")
gdf_bus_lanes = gdf_bus_lanes.set_crs("EPSG:4326")

In [34]:
# # Создаем траектории машин
# def create_trajectories(df):
#     trajs = []
#     for car_id, group in df.groupby('Car_ID'):
#         points = [Point(lon, lat) for lon, lat in zip(group['Longitude'], group['Latitude'])]
#         trajs.append({'Car_ID': car_id, "geometry": LineString(points)})
#     return gpd.GeoDataFrame(trajs, crs="EPSG:4326")

# gdf_trips = create_trajectories(df_trips)

In [62]:
# Добавляем buffer(0.0003) к выделенным полосам - примерно 30 метров
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=3857)  # Web Mercator (метры)
gdf_bus_lanes["geometry"] = gdf_bus_lanes["geometry"].buffer(30)  # 30 метров (0.0003 градусов ~ 30 м)
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=4326)  # Возвращаем в исходную систему

# Определяем пересечения с учетом buffer
df_trips["on_bus_lane"] = df_trips.intersects(gdf_bus_lanes["geometry"].union_all())

In [37]:
# Анализ скорости на участках
df_trips["next_time"] = df_trips.groupby('Car_ID')['Day_time'].shift(-1)
df_trips["next_lon"] = df_trips.groupby('Car_ID')['Longitude'].shift(-1)
df_trips["next_lat"] = df_trips.groupby('Car_ID')['Latitude'].shift(-1)

# Вычисляем расстояние между последовательными точками
df_trips['distance'] = df_trips.apply(
    lambda row: geodesic((row['Latitude'], row['Longitude']), (row["next_lat"], row["next_lon"])).meters / 1000
    if pd.notnull(row["next_lat"]) else None,
    axis=1
)

# Вычисляем время между последовательными точками
df_trips["time"] = df_trips.apply(
    lambda row: (row["next_time"] - row['Day_time']).total_seconds() / 3600 if pd.notnull(row["next_time"]) else None,
    axis=1
)

# Вычисляем сколрость между последовательными точками
df_trips["speed"] = df_trips.apply(
    lambda row: row["distance"] / row["time"] if pd.notnull(row["distance"]) and row["time"] not in [None, 0] else None,
    axis=1
)

In [38]:
df_trips.iloc[236:248]

,Car_ID,Day_time,Speed_gl,Ignition,Longitude,Latitude,next_time,next_lon,next_lat,distance,time,speed
236,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:27,15,1,55.767596,37.83425,2025-02-17 06:53:31,55.76751,37.834183,0.010612,0.001111,9.550852
237,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:31,7,1,55.76751,37.834183,2025-02-17 06:53:32,55.767491,37.834136,0.005478,0.000278,19.721727
238,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:32,14,1,55.767491,37.834136,2025-02-17 06:53:33,55.767483,37.834058,0.008686,0.000278,31.269875
239,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:33,19,1,55.767483,37.834058,2025-02-17 06:53:41,55.767438,37.832803,0.139353,0.002222,62.708824
240,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:41,39,1,55.767438,37.832803,2025-02-17 06:53:49,55.767428,37.83157,0.136858,0.002222,61.585909
241,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:49,21,1,55.767428,37.83157,2025-02-17 06:53:50,55.767401,37.831496,0.008551,0.000278,30.781843
242,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:50,18,1,55.767401,37.831496,2025-02-17 06:53:51,55.767373,37.831453,0.005372,0.000278,19.337944
243,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:51,14,1,55.767373,37.831453,2025-02-17 06:53:52,55.767338,37.831433,0.003798,0.000278,13.671161
244,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:52,15,1,55.767338,37.831433,2025-02-17 06:53:53,55.767295,37.831436,0.003800,0.000278,13.680149
245,77403 (М947ОК67) Renault Logan,2025-02-17 06:53:53,18,1,55.767295,37.831436,2025-02-17 06:53:54,55.767243,37.831481,0.006775,0.000278,24.390508


In [ ]:
# Средняя скорость на выделенных полосах и вне их
avg_speed_bus_lane = 50  # Можно получить из данных общественного транспорта

# Оценка выигрыша во времени
df_trips['predicted_time'] = df_trips.apply(
    lambda row: row['distance'] / avg_speed_bus_lane if row["on_bus_lane"] else row["distance"] / row["speed"],
    axis=1
)
df_trips["time_saved"] = df_trips["time"] - df_trips["predicted_time"]

# Итоговый выигрыш во времени
total_time_saved = df_trips["time_saved"].sum()

print(f"Общая экономия времени: {total_time_saved / 3600:.2f} часов")
